In [ ]:
from collections import deque

class Nodo:
    def __init__(self, valor):
        self.valor = valor
        self.siguiente = None

class ListaLigada:
    def __init__(self):
        self.cabeza = None

    def agregar(self, valor):
        nuevo = Nodo(valor)
        nuevo.siguiente = self.cabeza
        self.cabeza = nuevo

    def mostrar(self):
        actual = self.cabeza
        while actual:
            print(actual.valor)
            actual = actual.siguiente

class NodoBST:
    def __init__(self, nombre, telefono):
        self.nombre = nombre
        self.telefono = telefono
        self.izq = None
        self.der = None

class ArbolBST:
    def __init__(self):
        self.raiz = None

    def insertar(self, nombre, telefono):
        def _insertar(nodo, nombre, telefono):
            if nodo is None:
                return NodoBST(nombre, telefono)
            if nombre < nodo.nombre:
                nodo.izq = _insertar(nodo.izq, nombre, telefono)
            else:
                nodo.der = _insertar(nodo.der, nombre, telefono)
            return nodo

        self.raiz = _insertar(self.raiz, nombre, telefono)

    def eliminar(self, nombre):
        def _eliminar(nodo, nombre):
            if nodo is None:
                return None
            if nombre < nodo.nombre:
                nodo.izq = _eliminar(nodo.izq, nombre)
            elif nombre > nodo.nombre:
                nodo.der = _eliminar(nodo.der, nombre)
            else:
                # caso 1: 0 hijos
                if nodo.izq is None:
                    return nodo.der
                if nodo.der is None:
                    return nodo.izq
                # caso 2: 2 hijos → mínimo del subárbol derecho
                sucesor = nodo.der
                while sucesor.izq:
                    sucesor = sucesor.izq
                nodo.nombre, nodo.telefono = sucesor.nombre, sucesor.telefono
                nodo.der = _eliminar(nodo.der, sucesor.nombre)
            return nodo

        self.raiz = _eliminar(self.raiz, nombre)

    def inorden(self):
        def _inorden(nodo):
            if nodo:
                _inorden(nodo.izq)
                print(f"{nodo.nombre}: {nodo.telefono}")
                _inorden(nodo.der)
        _inorden(self.raiz)

    def contactos_en_lista(self):
        """Devuelve [(nombre, tel), ...] en orden alfabético."""
        resultado = []

        def _inorden(nodo):
            if nodo:
                _inorden(nodo.izq)
                resultado.append((nodo.nombre, nodo.telefono))
                _inorden(nodo.der)

        _inorden(self.raiz)
        return resultado

    def buscar_prefijo(self, prefijo):
        """Devuelve contactos cuyo nombre comienza con 'prefijo'."""
        resultado = []

        def _buscar(nodo):
            if not nodo:
                return
            # Recorremos todo (simple) – se podría podar, pero así es más claro
            _buscar(nodo.izq)
            if nodo.nombre.startswith(prefijo):
                resultado.append((nodo.nombre, nodo.telefono))
            _buscar(nodo.der)

        _buscar(self.raiz)
        return resultado

class TablaHash:
    def __init__(self):
        self.tabla = {}

    def agregar(self, nombre, telefono):
        self.tabla[nombre] = telefono

    def eliminar(self, nombre):
        if nombre in self.tabla:
            del self.tabla[nombre]

    def buscar(self, nombre):
        return self.tabla.get(nombre, None)

# ==============================
# Componentes globales
# ==============================
historial = ListaLigada()
arbol = ArbolBST()
tabla = TablaHash()

undo_stack = []               # stack para undo
recientes = deque(maxlen=5)   # cola de últimas búsquedas


# ==============================
# Helpers "raw" (sin registrar undo)
# ==============================
def _agregar_raw(nombre, telefono):
    tabla.agregar(nombre, telefono)
    arbol.insertar(nombre, telefono)

def _eliminar_raw(nombre):
    tel = tabla.buscar(nombre)
    if tel is None:
        return None
    tabla.eliminar(nombre)
    arbol.eliminar(nombre)
    return tel

def _modificar_raw(nombre, nuevo_tel):
    tabla.agregar(nombre, nuevo_tel)
    arbol.eliminar(nombre)
    arbol.insertar(nombre, nuevo_tel)


# ==============================
# Funciones públicas (con undo + historial)
# ==============================
def agregar_contacto(nombre, telefono):
    _agregar_raw(nombre, telefono)
    historial.agregar(f"Se agregó contacto: {nombre}")
    undo_stack.append(("agregar", nombre))
    print("✔ Contacto agregado.")

def eliminar_contacto(nombre):
    tel = tabla.buscar(nombre)
    if tel is None:
        print("✘ El contacto no existe.")
        return
    _eliminar_raw(nombre)
    historial.agregar(f"Se eliminó contacto: {nombre}")
    undo_stack.append(("eliminar", nombre, tel))
    print("✔ Contacto eliminado.")

def modificar_contacto(nombre, nuevo_tel):
    tel_anterior = tabla.buscar(nombre)
    if tel_anterior is None:
        print("✘ El contacto no existe.")
        return
    _modificar_raw(nombre, nuevo_tel)
    historial.agregar(f"Se modificó contacto: {nombre}")
    undo_stack.append(("modificar", nombre, tel_anterior))
    print("✔ Contacto modificado.")

def buscar_contacto(nombre):
    tel = tabla.buscar(nombre)
    recientes.append(nombre)
    historial.agregar(f"Búsqueda: {nombre}")
    if tel:
        print(f"✔ {nombre} → {tel}")
    else:
        print("✘ No encontrado.")

def undo():
    if not undo_stack:
        print("Nada que deshacer.")
        return

    accion = undo_stack.pop()

    tipo = accion[0]
    if tipo == "agregar":
        nombre = accion[1]
        _eliminar_raw(nombre)
        historial.agregar(f"UNDO: se deshizo agregar {nombre}")
        print("↩ Deshecho: agregar.")
    elif tipo == "eliminar":
        nombre, tel = accion[1], accion[2]
        _agregar_raw(nombre, tel)
        historial.agregar(f"UNDO: se deshizo eliminar {nombre}")
        print("↩ Deshecho: eliminar.")
    elif tipo == "modificar":
        nombre, tel_anterior = accion[1], accion[2]
        _modificar_raw(nombre, tel_anterior)
        historial.agregar(f"UNDO: se deshizo modificar {nombre}")
        print("↩ Deshecho: modificar.")

def mostrar_contactos():
    print("Contactos (orden alfabético):")
    arbol.inorden()

def mostrar_historial():
    print("Historial de acciones:")
    historial.mostrar()

def mostrar_recientes():
    print("Últimas búsquedas:")
    for n in recientes:
        print("•", n)

def buscar_por_prefijo():
    pref = input("Prefijo a buscar: ")
    resultados = arbol.buscar_prefijo(pref)
    if not resultados:
        print("No se encontraron contactos con ese prefijo.")
        return
    print(f"Contactos que comienzan con '{pref}':")
    for nombre, tel in resultados:
        print(f"{nombre}: {tel}")

def exportar_contactos(nombre_archivo):
    contactos = arbol.contactos_en_lista()
    try:
        with open(nombre_archivo, "w", encoding="utf-8") as f:
            for nombre, tel in contactos:
                f.write(f"{nombre},{tel}\n")
        historial.agregar(f"Se exportaron contactos a {nombre_archivo}")
        print("✔ Exportación completada.")
    except Exception as e:
        print("Error al exportar:", e)

def importar_contactos(nombre_archivo):
    try:
        with open(nombre_archivo, "r", encoding="utf-8") as f:
            for linea in f:
                linea = linea.strip()
                if not linea:
                    continue
                nombre, tel = linea.split(",", 1)
                agregar_contacto(nombre, tel)
        historial.agregar(f"Se importaron contactos desde {nombre_archivo}")
        print("✔ Importación completada.")
    except FileNotFoundError:
        print("✘ Archivo no encontrado.")
    except Exception as e:
        print("Error al importar:", e)


# ==============================
# Menú de consola
# ==============================
def menu():
    while True:
        print("\n===== AGENDA DE CONTACTOS =====")
        print("1. Agregar contacto")
        print("2. Buscar contacto")
        print("3. Eliminar contacto")
        print("4. Modificar contacto")
        print("5. Mostrar contactos ordenados")
        print("6. Mostrar historial")
        print("7. Deshacer (undo)")
        print("8. Últimas búsquedas")
        print("9. Buscar por prefijo")
        print("10. Exportar contactos a archivo")
        print("11. Importar contactos desde archivo")
        print("12. Salir")

        opcion = input("Elige una opción: ")

        if opcion == "1":
            n = input("Nombre: ")
            t = input("Teléfono: ")
            agregar_contacto(n, t)
        elif opcion == "2":
            n = input("Nombre: ")
            buscar_contacto(n)
        elif opcion == "3":
            n = input("Nombre: ")
            eliminar_contacto(n)
        elif opcion == "4":
            n = input("Nombre: ")
            t = input("Nuevo teléfono: ")
            modificar_contacto(n, t)
        elif opcion == "5":
            mostrar_contactos()
        elif opcion == "6":
            mostrar_historial()
        elif opcion == "7":
            undo()
        elif opcion == "8":
            mostrar_recientes()
        elif opcion == "9":
            buscar_por_prefijo()
        elif opcion == "10":
            archivo = input("Nombre de archivo para exportar (ej. contactos.csv): ")
            exportar_contactos(archivo)
        elif opcion == "11":
            archivo = input("Nombre de archivo para importar: ")
            importar_contactos(archivo)
        elif opcion == "12":
            print("Adiós :)")
            break
        else:
            print("Opción no válida.")

if __name__ == "__main__":
    menu()